# RAG

Retrieval-Augmented Generation (RAG) é uma técnica que combina modelos de linguagem com mecanismos de recuperação de informações para melhorar a geração de texto.

O RAG revolucionou a interação, compreensão e geração de linguagem humana pelos sistemas de IA. Tornou os modelos de linguagem mais versáteis e inteligentes, sendo crucial para chatbots sofisticados e ferramentas complexas de criação de conteúdo.
Uma das aplicações mais poderosas dos LLMs é a criação de chatbots sofisticados de perguntas e respostas (Q&A). Esses chatbots podem responder a perguntas sobre informações específicas usando RAG.

> [ ver slides para mais sobre RAG ]

O LangChain tem vários componentes projetados para ajudar a criar aplicativos de perguntas e respostas e aplicativos RAG de forma mais geral.

## Importações

In [1]:
import torch
import os
from langchain_groq import ChatGroq
import time

from langchain.prompts import PromptTemplate
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.messages import SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_pinecone import PineconeVectorStore
#from langchain_community.vectorstores import Pinecone as PineconeVectorStore

/home/kevin/anaconda3/envs/langchain/lib/python3.13/site-packages/langchain_pinecone/__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


In [2]:
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

True

## Carregando a LLM


In [3]:
llm = ChatGroq(model_name='llama-3.3-70b-versatile',temperature=0)


## Prompt para RAG

Para implementar o RAG, devemos reservar um espaço no template do prompt para que seja alocado nessa parte o contexto que queremos usar

Basearemos nesse template que está hospedado no Hub do LangSmith https://smith.langchain.com/hub/rlm/rag-prompt É um prompt bastante usado para esse objetivo

Mais tarde ensinaremos como puxar diretamente os prompts de lá sem precisar copiar e colar. Mas agora, fazemos assim pois queremos adequar ao template do modelo Llama 3

In [4]:
template_rag = """
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
Você é um assistente virtual prestativo e está respondendo perguntas gerais.
Use os seguintes pedaços de contexto recuperado para responder à pergunta.
Se você não sabe a resposta, apenas diga que não sabe. Mantenha a resposta concisa.
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Pergunta: {pergunta}
Contexto: {contexto}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

In [5]:
prompt_rag = PromptTemplate.from_template(template_rag)
print(prompt_rag)

input_variables=['contexto', 'pergunta'] input_types={} partial_variables={} template='\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente virtual prestativo e está respondendo perguntas gerais.\nUse os seguintes pedaços de contexto recuperado para responder à pergunta.\nSe você não sabe a resposta, apenas diga que não sabe. Mantenha a resposta concisa.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\nPergunta: {pergunta}\nContexto: {contexto}\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n'


## Aplicação para RAG com contextos maiores

> [ Conferir slides para explicações das etapas ]

Nosso próximo exemplo vai consistir de acessar uma página na internet e usar RAG para fazer a LLM "conversar" com ela, usando o seu conteúdo como contexto e assim responder nossas dúvidas. Ou seja, basicamente estaremos fazendo um *Webscraping* e utilizando o conteúdo lido como contexto

A informação que queremos saber seria impossível com esse modelo que estamos usando nesse exemplo (llama 3.0) pois os dados de treinamento mais recentes são de 2023, portanto, não teria como saber nada do que aconteceu em 2025. Isso significa que o modelo vai retornar algo como "não aconteceu ainda" ou "não tenho como saber esse tipo de informação". Em alguns casos, dependendo do prompt, e ele pode tentar responder algo plausível mas levemente alucinado - portanto ainda não seria totalmente coerente com nossa dúvida

Mesmo que fosse uma informação anterior aos dados de treinamento, ainda assim o RAG poderia ser usado, pelas razões que comentamos, principalmente por garantir resultados mais precisos


In [6]:

contexto = ""
print(contexto)
chain = prompt_rag | llm | StrOutputParser()

pergunta = "Quais as atrizes indicadas para Oscar 2025"

res = chain.invoke({"pergunta": pergunta, "contexto": contexto})
res

'Não sei, pois não tenho informações atualizadas sobre as indicações para o Oscar 2025.'


## Etapas de Indexação





### 1 - Carregar o conteúdo

Primeiramente, precisamos carregar o conteúdo desejado.
Como nesse exemplo queremos acessar uma página web (que contém as informações de nossa dúvida)

Para isso:

* Usamos o DocumentLoaders, que são objetos que carregam dados de uma fonte e retornam uma lista de `Documents`. Nesse contexto, um Document é um objeto com `page_content` (retornado em string) e `metadata` (retornado em dicionário).

O LangChain possui suporte a centenas de formas de carregadores de documentos, para os mais diversos meios e formatos ou extensões de arquivos (PDF, CSV, etc.)

> Confira aqui: https://python.langchain.com/v0.2/docs/integrations/document_loaders/

* Como queremos carregar os dados de uma página web nós importamos o WebBaseLoader, que usa urllib para carregar HTML de URLs e a biblioteca BeautifulSoup para converter em texto.



In [7]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_huggingface import HuggingFaceEmbeddings
from pinecone import Pinecone, ServerlessSpec
from langchain_chroma import Chroma

USER_AGENT environment variable not set, consider setting it to identify your requests.


* Você pode customizar a conversão HTML para texto passando parâmetros para o parser do BeautifulSoup via bs_kwargs.

  * Poderiamos especificar tags HTML com classes específicas, para puxar apenas o conteúdo que está dentro dela - nesse caso, poderia ser o bloco HTML que contém o conteúdo do artigo, que é onde estão as informações relevantes - e então todas as outras partes da página são removidas (que contém informações não relevantes, como cabeçalho do site, rodapé, etc.).
  
  * Para descobrir a classe: selecione a opção "Inspecionar elemento" que aparece ao clicar com o botão direito na página. ou "Ferramentas de desenvolvedor > Inspecionar" (Ctrl + Shift + I)

Neste caso, não usaremos pois é uma página que não tem tanto conteúdo fora do container principal do site, portanto não acaba sendo um problema.

Mas você pode filtrar com base nas classes para que pegue somente o texto dentro do bloco principal de conteúdo da página


In [8]:
loader = WebBaseLoader(web_paths = ("https://www.bbc.com/portuguese/articles/c05m763gq34o",),)
docs = loader.load()

In [9]:
len(docs[0].page_content)

9549

Acima usamos o len() para exibir comprimento (em caracteres) do conteúdo da primeira página carregada

Podemos dar print no docs, o que mostrará o conteúdo completo.

Usamos isso para verificar a quantidade de conteúdo carregado e visualizar uma amostra inicial do texto carregado. Isso é útil para garantir que o carregamento do conteúdo e a filtragem dos elementos HTML relevantes foram realizados corretamente

In [10]:
docs

[Document(metadata={'source': 'https://www.bbc.com/portuguese/articles/c05m763gq34o', 'title': 'Oscar 2025: a lista com todos os vencedores - BBC News Brasil', 'description': "Indicado a três categorias, 'Ainda Estou Aqui' garantiu estatueta de Melhor Filme Estrangeiro.", 'language': 'pt-br'}, page_content='Oscar 2025: a lista com todos os vencedores - BBC News BrasilBBC News, BrasilVá para o conteúdoSeçõesNotíciasBrasilInternacionalEconomiaSaúdeCiênciaTecnologiaVídeosPodcastsNotíciasBrasilInternacionalEconomiaSaúdeCiênciaTecnologiaVídeosPodcastsOs grandes vencedores do Oscar 2025Crédito, Getty Images/ReutersLegenda da foto, Walter Salles, Mikey Madison e Adrien Brody levaram estatuetas por categorias de Melhor Filme Internacional, Melhor Atriz e Melhor Ator, respectivamente2 março 2025O Brasil ganhou sua primeira estatueta da história neste domingo (02/03) na 97ª edição do Oscar, o prêmio mais importante do cinema mundial. Ainda Estou Aqui, dirigido por Walter Salles, foi o vencedor n

A linha de baixo imprime os primeiros 300 caracteres do conteúdo da primeira página carregada. A sintaxe [:300] é usada para obter uma substring dos primeiros 300 caracteres da string page_content. Pode ser uma maneira rápida de verificação

In [11]:
print(docs[0].page_content[:300])

Oscar 2025: a lista com todos os vencedores - BBC News BrasilBBC News, BrasilVá para o conteúdoSeçõesNotíciasBrasilInternacionalEconomiaSaúdeCiênciaTecnologiaVídeosPodcastsNotíciasBrasilInternacionalEconomiaSaúdeCiênciaTecnologiaVídeosPodcastsOs grandes vencedores do Oscar 2025Crédito, Getty Images/


### 2 - Divisão em pedaços de texto / Split

Nosso documento carregado tem mais de 10 mil caracteres, o que seria muito grande para passar como contexto usando o método que fizemos até então.

Por exemplo pro GPT-4, o tamanho da janela de contexto do é de cerca de 8.000 tokens, o que equivale aproximadamente a 32.000 caracteres. Então na verdade para esse exemplo até poderia caber a postagem completa em sua janela de contexto, porém as LLMs geralmente terão dificuldade em localizar informações dentro de entradas tão longas.

No entanto, verá que é bem fácil um documento passar esse limite de tokens, o que fará com que o modelo não consiga processar tudo, já que é uma sequência de texto muito longa para caber na janela de contexto da grande maioria dos modelos.

Portanto, é uma boa prática fazer a divisão em documentos tão longos, assim já temos o código pronto e preparado para conseguir processar documentos muito maiores.

> Processo de divisão

* Dividiremos o `Document` em pedaços para incorporação e armazenamento vetorial. Isso deve nos ajudar a recuperar apenas os bits mais relevantes da postagem do blog em tempo de execução.

* Usamos o RecursiveCharacterTextSplitter, que dividirá recursivamente o documento usando separadores comuns, como novas linhas, até que cada pedaço tenha o tamanho apropriado. Este é o divisor de texto recomendado para casos de uso de texto gerais ou genérico.

* Neste caso, dividiremos em pedaços (chunks) de 1000 caracteres. Para isso usaremos o parâmetro `chunk_size`. Um tamanho de chunk menor resultará em mais pedaços, enquanto um tamanho de chunk maior resultará em menos pedaços.

* Com a divisão, é necessário fazer o que chamamos de sobreposição (overlap). Essa sobreposição ajuda a evitar que uma declaração seja separada do seu contexto importante. Aqui nesse exemplo vamos definir 200 caracteres de sobreposição entre os pedaços - usamos o parâmetro `chunk_overlap`.  Uma sobreposição de chunks maiores resultará em mais pedaços compartilhando caracteres comuns, enquanto uma sobreposição de chunks menores resultará em menos pedaços compartilhando caracteres comuns.

 * Escolhendo o valor de `chunk_size` e o `chunk_overlap` - geralmente recomenda-se experimentar diferentes valores, pois depende do problema específico que você está tentando resolver. No entanto, em geral, é uma boa ideia usar um tamanho de chunk/bloco pequeno para tarefas que exigem uma visão detalhada do texto (fine-grained view) e um tamanho de bloco maior para tarefas que exigem uma visão mais "holística" do texto (ou seja, uma visão mais geral, do todo) .


* E definimos `add_start_index=True` para que o índice de caracteres no qual cada Documento dividido começa dentro do Documento inicial seja preservado como atributo de metadados "start_index".

> Para outros tipos de transformações possíveis de serem feitos com a biblioteca LangChain, consulte a documentação https://python.langchain.com/v0.2/docs/integrations/document_transformers/

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200, add_start_index = True)
chunks = text_splitter.split_documents(docs)

In [ ]:
len(chunks)

In [ ]:
chunks[1]

Podemos trabalhar com metadata usando RAG por exemplo para rastrear onde a LLM obteve a resposta (útil caso estejamos usando várias fontes/arquivos diferentes para RAG)

In [ ]:
chunks[1].metadata

> Mais sobre os text splitters https://python.langchain.com/v0.2/docs/how_to/#text-splitters

### 3 - Armazenamento

Agora precisamos indexar nossos pedaços de texto para que possamos pesquisá-los. A maneira mais comum de fazer isso é incorporar o conteúdo de cada divisão de document e inserir esses embeddings em um banco de dados de vetores (ou armazenamento de vetores).

Quando queremos pesquisar em nossas divisões, pegamos uma consulta de pesquisa de texto, a incorporamos e realizamos algum tipo de pesquisa de "similaridade" para identificar as divisões armazenadas com os embeddings mais semelhantes ao nosso embedding de consulta.

A medida de similaridade mais simples é a similaridade de cosseno — medimos o cosseno do ângulo entre cada par de embeddings (que são vetores de alta dimensão).


#### Embeddings

Embedding é uma representação numérica de um texto. Olhando para eles, não são nada além de números, mas por trás dos panos, eles possuem uma relação entre si. Isso significa que palavras ou frases com significados semelhantes terão embeddings próximos uns dos outros em um espaço vetorial. Através dos embeddings conseguiremos procurar por itens similares (nesse caso, palavras)

> [ Mais explicações nos slides com título "Embeddings" ]

Podemos incorporar e armazenar todas as nossas divisões de documentos em um único comando usando o armazenamento de vetores Chroma

Mas antes, precisamos escolher qual embedding usar.

> Modelos de embedding open source
 * A principal vantagem de uso de modelo é que, assim como a LLM, modelo open source podemos rodar de graça e localmente offline

 * No Hugging Face podemos encontrar vários:
  * https://huggingface.co/models?sort=trending&search=embeddings
  * https://huggingface.co/models?pipeline_tag=sentence-similarity&sort=trending

 * Vamos escolher o "sentence-transformers/all-mpnet-base-v2", que costuma ser bom e funcionou bem para nossos resultdos.

 * Mas você pode testar outros também se desejar.

> Modelos de embedding proprietários

 * Você també pode encontrar modelos de embedding em soluções proprietárias com o OpenAI, que disponibiliza também modelo de embedding como o text-embedding-3-large (mais info, consulte [aqui](https://platform.openai.com/docs/guides/embeddings)). O valor do modelo de embedding é bem mais barato, para consultar veja [openai.com/api/pricing/](https://openai.com/api/pricing/)

 * Caso opte por essa opção, pode pular essa parte da definição da variável contendo o modelo de embeddings, pule direto para o código que faz o armazenamento como Chroma, onde é definido o OpenAIEmbeddings().

> Além do HuggingFaceEmbeddings e do OpenAIEmbeddings, há várias outras classes disponíveis para você implementar outros tipos de modelos de embedding, consulte: https://python.langchain.com/v0.2/docs/integrations/text_embedding/



In [ ]:
hf_embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2")

In [ ]:
input_test = "Um teste apenas"
result = hf_embeddings.embed_query(input_test)

In [ ]:
len(result)

In [ ]:
print(result)

#### Armazenando no banco de dados vetorial

Nós estamos escolhendo o **Chroma** como nosso banco de dados de vetores por ser bastante usado e versátil.

Assim como os outros componentes, o wrapper VectorStore possui diversas outras classes além do Chroma, assim você pode armazenar os vetores usando outro método ou serviço de sua preferência: https://python.langchain.com/v0.2/docs/integrations/vectorstores/

* Outras opções: FAISS, Pinecone, Qdrant, etc.

## PINECONE

In [ ]:

pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))


In [ ]:
index_name = "langchain-index" 
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()] # change if desired

In [ ]:
# Deletar e recriar o índice para garantir que a dimensão esteja correta
if index_name in existing_indexes:
    print(f"Deletando o índice existente '{index_name}'...")
    pc.delete_index(index_name)
    time.sleep(1) # Aguardar a exclusão

In [ ]:
pc.create_index(
    name=index_name,
    metric="cosine",
    dimension=768, # Isso irá retornar 768
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)
while not pc.describe_index(index_name).status["ready"]:
    print("Aguardando o índice ficar pronto..."+pc.describe_index(index_name).status)
    time.sleep(1)

index = pc.Index(index_name)

In [ ]:
vectorstore = PineconeVectorStore.from_documents(
    documents=chunks,
    embedding=hf_embeddings,
    index_name=index_name,
)


Isso conclui a parte de Indexação de nessa pipeline de RAG.

Até este ponto, já temos um repositório de vetores consultável contendo os conteúdos em blocos.

Dada uma pergunta feita, devemos ser capazes de retornar os snippets de conteúdo que respondem à pergunta.

## Etapas de Recuperação e geração



### 4 - Configurando o recuperador de texto / Retriever

Nessa aplicação que estamos montando, queremos que receba uma pergunta do usuário, busque documentos relevantes para essa pergunta, passe os documentos recuperados e a pergunta inicial para um modelo e retorne uma resposta.

Primeiro, precisamos definir nossa lógica para buscar nos documentos. O LangChain define uma interface chamada Retriever, que envolve um índice capaz de retornar `Documents` relevantes dado uma consulta em forma de string.

O tipo mais comum de Retriever é o [VectorStoreRetriever](https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStoreRetriever.html), que utiliza as capacidades de busca por similaridade de um vector store para facilitar a recuperação.

Qualquer VectorStore pode ser facilmente transformado em um Retriever com `VectorStore.as_retriever()`. Quanto aos parâmetros:

* search_type - é o tipo de pesquisa que será realizada. Lembrando que o que estamos buscando basicamente é similaridade, dentro do documento, como uma página web ou pdf por exemplo

* search_kwargs: "k" - Também podemos limitar o número de documentos k retornados pelo recuperador. Aqui você pode testar bastante os valores e verificar os resultados. 6 é um valor considerado meio padrão - e que funcionou bem para nossos testes - mas você pode variar ele para mais ou para menos, testar qual valor fica melhor pro seu caso
(aqui numero menor como 3 não deu conta de algumas perguntas, você pode diminuir e testar para comprovar)







In [ ]:
retriever = vectorstore.as_retriever(search_type = "similarity", search_kwargs={"k": 6})

> Explorando mais os retrievers  

* Outros métodos de busca - Por padrão, o retriever de armazenamento de vetores usa pesquisa de similaridade. Se o armazenamento de vetores que estiver usando suportar pesquisa de "relevância marginal máxima" (baseado no MMR - Maximum marginal relevance retrieval), você pode especificar isso como o tipo de pesquisa, basta trocar "similarity" por "mmr"

 * O [MMR](https://www.cs.cmu.edu/~jgc/publication/The_Use_MMR_Diversity_Based_LTMIR_1998.pdf) (Relevância Marginal Máxima) seleciona por relevância e diversidade entre os documentos recuperados para evitar passar em contexto duplicado. Ou seja, é um método usado para evitar redundância ao recuperar itens relevantes para uma consulta. Em vez de simplesmente recuperar os itens mais relevantes (que muitas vezes podem ser muito semelhantes entre si), o MMR garante um equilíbrio entre relevância e diversidade nos itens recuperados.

* MultiQueryRetriever - permite gerar variações da pergunta de entrada para melhorar a taxa de acerto de recuperação ([mais info](https://python.langchain.com/v0.2/docs/how_to/MultiQueryRetriever/))

* MultiVectorRetriever - já esse gera variantes dos embeddings, também para melhorar a taxa de acerto de recuperação ([mais info](https://python.langchain.com/v0.2/docs/how_to/multi_vector/))

### 5 - Geração

Vamos juntar tudo em uma chain, que irá: pegar uma pergunta -> recupera documentos relevantes -> construir um prompt -> passar para o modelo -> processar a saída.



In [ ]:
template_rag

In [ ]:
prompt_rag = PromptTemplate(
    input_variables=["contexto", "pergunta"],
    template=template_rag,
)
prompt_rag

> Formação de documentos

A função abaixo recebe uma lista de documentos (docs) e retorna uma única string onde o conteúdo de cada documento é concatenado com duas quebras de linha entre eles. No contexto de processamento de texto para uma pipeline de RAG, essa função formata os documentos recuperados de modo que seu conteúdo possa ser passado de maneira estruturada para um modelo de linguagem, facilitando a geração de respostas baseadas nos dados dos documentos

In [ ]:
def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

> Definição da chain

Podemos deixar a chain desse modo, com quebra de linha, pois dará mais legibilidade (também graças à sintaxe do LCEL, conforme citado) já que agora temos mais elos / componentes em nossa chain

Aqui usamos o método RunnablePassthrough, que por si só permite que você passe entradas inalteradas

In [ ]:
chain_rag = ({"contexto": retriever | format_docs, "pergunta": RunnablePassthrough()}
             | prompt_rag
             | llm
             | StrOutputParser())

> Geração  

In [ ]:
chain_rag.invoke("Quais atrizes foram indicadas para o oscar 2025?")

> Limpando o vector store


O comando vectorstore.delete_collection() é usado no contexto de gerenciamento de um repositório de vetores, que em sua aplicação é uma estrutura semelhante a um banco de dados que armazena representações vetoriais de dados de texto



* Este comando instrui o vectorstore a deletar toda a coleção de dados que ele contém. Uma coleção refere-se ao conjunto de todos os documentos (trechos de texto) e suas representações vetoriais correspondentes que foram indexados e armazenados no vectorstore.

* Observação sobre impacto na aplicação: Ao executar vectorstore.delete_collection(), você remove efetivamente todos os dados indexados do repositório de vetores. Esta ação limpa o banco de dados, o que pode ser necessário durante redefinições do sistema, atualizações ou quando você deseja limpar dados antigos para abrir espaço para novos dados a serem processados e armazenados.

Esta função é crítica para manter a limpeza e a relevância dos dados em aplicações que dependem de conjuntos de dados dinâmicos ou que requerem atualizações periódicas na sua estrutura de informações. Ela garante que o sistema possa ser atualizado ou reestruturado sem que dados residuais de operações anteriores interfiram nas novas operações.

### Informações extras

* Para outros tipos de documentos, consulte os métodos de [Document loaders](https://python.langchain.com/v0.2/docs/integrations/document_loaders/)

 * Para leitura de PDF por exemplo, podemos usar o PyPDFLoader. Se esse for o seu objetivo, basta mudar a etapa 1 do processo descrito acima, para que ao invés do WebBaseLoader use o PyPDFLoader (e passe o arquivo desejado). Teremos um projeto inteiro dedicado à leitura de RAG com arquivos  

* Ao ler dados da internet com Webscraping podemos nos deparar com algumas páginas onde não pôde ser lido devido à limitações dessa técnica com as bibliotecas convencionais

 > Dica: usar um serviço para isso com o FireCrawl - tem um uso gratuito mas também oferece planos pagos

 * Para não ter problemas com o web scrapping é recomendado usar ele.
Como é de se imaginar, ele se integra bem ao LangChain https://python.langchain.com/v0.2/docs/integrations/document_loaders/firecrawl/

## Resumo sobre RAG e LLMs usando LangChain

Resumo geral do pipeline que montamos acima:

1. Carregar o conteúdo da página ou PDF ou outro arquivo/mídia
2. Divisão em Chunks: O primeiro passo é dividir os documentos em pequenos pedaços, ou chunks.
3. Armazenamento e Transformação em Embeddings: Esses chunks são transformados em embeddings, que são representações vetoriais dos textos. Os embeddings são armazenados em um banco de dados vetorial (vector database).
4. Uso de Retriever: O banco de dados vetorial fornece um retriever (recuperador de informações) que busca os chunks mais relevantes com base em um algoritmo de similaridade.
5. Junção do contexto ao prompt e geração do resultado final

**Fluxo de Trabalho:**
* Primeira Chain (LLM + Prompt): O modelo de linguagem (LLM) recebe um prompt inicial e gera uma resposta.
* Integração com Retriever: A resposta inicial é combinada com o retriever para formar outro elo na cadeia.

**Funcionamento do Sistema:**
* Input como Embedding: O input fornecido é convertido em um embedding.
* Busca no banco de dados: O embedding do input é usado para buscar os chunks mais relevantes no banco de dados.
* Execução do LLM: Os chunks encontrados são utilizados pelo LLM para gerar a resposta final.
* Rastreamento de chunks utilizados: É possível acessar quais chunks foram usados na geração da resposta.

